In [ ]:
# Install the Redis Python client
!pip install redis

# Install the Redis server
# For Debian-based systems like Ubuntu (which Colab uses)
!apt-get update
!apt-get install -y redis-server

In [2]:
# Start the Redis server in the background
import os
import subprocess
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.0 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy

In [3]:
# Check if Redis server is already running
def is_redis_running():
    try:
        # Attempt to ping the Redis server
        subprocess.run(["redis-cli", "ping"], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return True
    except subprocess.CalledProcessError:
        return False

if not is_redis_running():
    # Start Redis server with default configuration
    os.system("redis-server --daemonize yes")
    # Wait for the server to start
    time.sleep(2)
    if is_redis_running():
        print("Redis server started successfully.")
    else:
        print("Failed to start Redis server.")
else:
    print("Redis server is already running.")

# Import the Redis module
import redis

# Connect to Redis
try:
    client = redis.Redis(host='localhost', port=6379, db=0)
    # Test the connection
    client.ping()
    print("Connected to Redis successfully.")
except redis.ConnectionError:
    print("Failed to connect to Redis.")

# Initialize user data
users = ["user:1001", "user:1002", "user:1003", "user:1004"]

# 1. Simulate user visits and increment the counter
def simulate_user_visits(user_id):
    client.incr(user_id)
    visit_count = client.get(user_id).decode("utf-8")
    print(f"User {user_id} has {visit_count} visits.")

# 2. Bulk simulate multiple user visits
def simulate_bulk_visits():
    for user_id in users:
        visits = client.incrby(user_id, 5)  # Increment each user's visit by 5
        print(f"User {user_id} has {visits} visits.")

# 3. Generate a leaderboard based on visits
def generate_leaderboard():
    print("\n=== User Visit Leaderboard ===")
    leaderboard = {}
    for user_id in users:
        count = client.get(user_id)
        if count is not None:
            leaderboard[user_id] = int(count.decode("utf-8"))
        else:
            leaderboard[user_id] = 0
    sorted_leaderboard = sorted(leaderboard.items(), key=lambda x: x[1], reverse=True)
    for rank, (user, count) in enumerate(sorted_leaderboard, start=1):
        print(f"{rank}. {user} - Visits: {count}")

# 4. Demonstrate deleting a user's counter
def reset_user_counter(user_id):
    client.delete(user_id)
    print(f"\nUser {user_id}'s counter has been reset.")

# Demo Process
print("1. Simulate a single user visit:")
simulate_user_visits("user:1001")

print("\n2. Bulk simulate user visits:")
simulate_bulk_visits()

print("\n3. Generate leaderboard:")
generate_leaderboard()

print("\n4. Delete a user's counter:")
reset_user_counter("user:1001")

print("\n5. Check leaderboard after deletion:")
generate_leaderboard()


Redis server started successfully.
Connected to Redis successfully.
1. Simulate a single user visit:
User user:1001 has 1 visits.

2. Bulk simulate user visits:
User user:1001 has 6 visits.
User user:1002 has 5 visits.
User user:1003 has 5 visits.
User user:1004 has 5 visits.

3. Generate leaderboard:

=== User Visit Leaderboard ===
1. user:1001 - Visits: 6
2. user:1002 - Visits: 5
3. user:1003 - Visits: 5
4. user:1004 - Visits: 5

4. Delete a user's counter:

User user:1001's counter has been reset.

5. Check leaderboard after deletion:

=== User Visit Leaderboard ===
1. user:1002 - Visits: 5
2. user:1003 - Visits: 5
3. user:1004 - Visits: 5
4. user:1001 - Visits: 0


In [8]:
#Single user
client = redis.StrictRedis(host='localhost', port=6379, db=0)

user_id = "user:12345"

client.incr(user_id)

visit_count = client.get(user_id).decode("utf-8")
print(f"User {user_id} Has: {visit_count} Visits")


User user:12345 Has: 4 Visits
